In [55]:
import PIL
from tkinter import *
from tkinter import ttk
from PIL import Image
from PIL import ImageTk
from PIL import ImageFilter
from PIL import ImageDraw, ImageOps
from tkinter import filedialog
import cv2
from skimage import filters
import numpy as np
from skimage import io, color, filters, transform, exposure
from scipy import signal, sparse

In [56]:
def upload():
    global panelA, panelB, image
    f_types = [('Jpg Files', '*.jpg'),('PNG Files','*.png')] 
    path = filedialog.askopenfilename(filetypes=f_types)
    image = Image.open(path) 
    image = np.array(image)
    height = image.shape[0]
    width = image.shape[1]
    aspect_ratio = width/height
    new_height = 700
    new_width = int(aspect_ratio * new_height)
    image = cv2.resize(image, (new_width,new_height), interpolation = cv2.INTER_AREA)

    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # image = Image.fromarray(image)
    image1 = Image.fromarray(image)

    image1 = ImageTk.PhotoImage(image1)

    panelA = Label(image=image1, relief="sunken")
    panelA.image = image1
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    
    return image

Change brightness

In [57]:
def change_intensity(channel, level):
    m = channel.shape[0]
    n = channel.shape[1]
    out = np.zeros_like(channel)
    for i in range(m):
        for j in range(n):
            new_brightness = int(channel[i, j] * (1+level))
            if(new_brightness > 255):
                out[i, j] = 255
            else:
                out[i, j] = new_brightness
    return out

In [58]:
def scale_pixel_value (color, brightnessFactor):
    scaledValue = float(color * (1+brightnessFactor))
    if scaledValue < 0:
        return 0
    elif scaledValue > 255:
        return 255
    return int(scaledValue)


def change_brightness():
    brightnessFactor = int(BRIGTHNESS_LEVEL.get())
    brightnessFactor = brightnessFactor / 100
    im = Image.fromarray(image)
    width, height = im.size
    out = Image.new('RGB', im.size, 0xffffff)
    for i in range(width):
        for j in range(height):
            r,g,b = im.getpixel((i,j))
            
            updatedR = scale_pixel_value(r, brightnessFactor)
            updatedG = scale_pixel_value(g, brightnessFactor)
            updatedB = scale_pixel_value(b, brightnessFactor)

            out.putpixel((i,j), (updatedR, updatedG, updatedB))
    out = ImageTk.PhotoImage(out)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out  
    

Change contrast

In [59]:

def change_contrast():
    img = Image.fromarray(image)
    level = int(CONTRAST_LEVEL.get())
    factor = (259 * (level + 255)) / (255 * (259 - level))
    def contrast(c):
        return 128 + factor * (c - 128)
    out = img.point(contrast)
    out = ImageTk.PhotoImage(out)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out


Change saturation

In [60]:
def change_saturation():
    level = int(SATURATION_LEVEL.get())
    level = level / 100
    im = np.array(image)
    hsv_img = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)

    hsv_img[:,:,1] = change_intensity(hsv_img[:,:,1], level)

    rgb_img = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)
    output = Image.fromarray(rgb_img.astype(np.uint8), 'RGB')
    out = ImageTk.PhotoImage(output)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out


Change hue

In [61]:
def change_hue():
    level = int(HUE_LEVEL.get())
    level = level / 100
    im = np.array(image)
    hsv_img = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)

    hsv_img[:,:,0] = change_intensity(hsv_img[:,:,1], level)

    rgb_img = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)
    output = Image.fromarray(rgb_img.astype(np.uint8), 'RGB')
    out = ImageTk.PhotoImage(output)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out

Blur

In [62]:
def filter_channel(im, D0):
    F = np.fft.fft2(im)
    Fshift = np.fft.fftshift(F) 
    M,N = im.shape
    H = np.zeros((M,N), dtype=np.float32)
    for u in range(M):
        for v in range(N):
            D = np.sqrt((u-M/2)**2 + (v-N/2)**2)
            H[u,v] = np.exp(-D**2/(2*D0*D0))
    Gshift = Fshift * H
    G = np.fft.ifftshift(Gshift)
    g = np.abs(np.fft.ifft2(G))
    return g

def filter_blur():
    level = int(BLUR_LEVEL.get())
    
    im = np.array(image)
    D0 = (im.shape[0]//2) * (11 - level) / 100

    out_r = filter_channel(im[:,:,0], D0)
    out_g = filter_channel(im[:,:,1], D0)
    out_b = filter_channel(im[:,:,2], D0)
    output = np.stack([out_r, out_g, out_b], axis=2)
    output = Image.fromarray(np.array(output).astype(np.uint8), 'RGB')
    out = ImageTk.PhotoImage(output)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out

Sharpening

In [63]:
def sharpen_channel(im):
    im = im / 255
    F = np.fft.fft2(im)
    Fshift = np.fft.fftshift(F) 
    P,Q = F.shape
    H = np.zeros((P,Q), dtype=np.float32)
    for u in range(P):
        for v in range(Q):
            H[u,v] = -4*np.pi*np.pi*((u-P/2)**2 + (v-Q/2)**2)

    Lap = H * Fshift
    Lap = np.fft.ifftshift(Lap)
    Lap = np.real(np.fft.ifft2(Lap))

    # convert the Laplacian Image value into range [-1,1]
    OldRange = np.max(Lap) - np.min(Lap)
    NewRange = 1 - -1
    LapScaled = (((Lap - np.min(Lap)) * NewRange) / OldRange) + -1
    c = -1
    g = im + c*LapScaled
    g = np.clip(g, 0, 1)
    return g

def sharpen():
    im = np.array(image)
    out_r = sharpen_channel(im[:,:,0])
    out_g = sharpen_channel(im[:,:,1])
    out_b = sharpen_channel(im[:,:,2])
    output = np.stack([out_r, out_g, out_b], axis=2)
    output = Image.fromarray((output*255).astype(np.uint8), 'RGB')
    out = ImageTk.PhotoImage(output)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out

Grayscale

In [64]:
def grayscale():
    im = Image.fromarray(image)
    width, height = im.size
    out = Image.new('RGB', im.size, 0xffffff)
    for i in range(width):
        for j in range(height):
            r,g,b = im.getpixel((i,j))

            tr = int(0.2989*r + 0.5870*g + 0.1140*b)
            tg = tr
            tb = tr

            if tr > 255:
                updatedR = 255 
            else:
                updatedR = tr

            if tg > 255:
                updatedG = 255 
            else:
                updatedG = tg

            if tb > 255:
                updatedB = 255
            else:
                updatedB = tb

            out.putpixel((i,j), (updatedR, updatedG, updatedB))
            
    out = ImageTk.PhotoImage(out)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out   



# def grayscale():
#     grayimg= cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     grayimg1= Image.fromarray(grayimg)
    
#     grayimg1= ImageTk.PhotoImage(grayimg1)
    
#     panelB = Label(image=grayimg1, borderwidth=5, relief="sunken")
#     panelB.image = grayimg1
#     panelB.grid(row= 1, column=4 , rowspan= 13,columnspan= 3, padx=20, pady=20)
    
#     return grayimg

Sepia

In [65]:

def sepia():
    im = Image.fromarray(image)
    width, height = im.size
    out = Image.new('RGB', im.size, 0xffffff)
    for i in range(width):
        for j in range(height):
            r,g,b = im.getpixel((i,j))

            tr = int(0.393*r + 0.769*g + 0.189*b)
            tg = int(0.349*r + 0.686*g + 0.168*b)
            tb = int(0.272*r + 0.534*g + 0.131*b)

            if tr > 255:
                updatedR = 255 
            else:
                updatedR = tr

            if tg > 255:
                updatedG = 255 
            else:
                updatedG = tg

            if tb > 255:
                updatedB = 255
            else:
                updatedB = tb

            out.putpixel((i,j), (updatedR, updatedG, updatedB))
    out = ImageTk.PhotoImage(out)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out

Inverse

In [66]:
def inversePixel(color):
    L = 255
    out = L - 1 - color
    return out

def inverse_image():
    im = Image.fromarray(image)
    width, height = im.size
    out = Image.new('RGB', im.size, 0xffffff)
    for i in range(width):
        for j in range(height):
            r,g,b = im.getpixel((i,j))
            
            updatedR = inversePixel(r)
            updatedG = inversePixel(g)
            updatedB = inversePixel(b)

            out.putpixel((i,j), (updatedR, updatedG, updatedB))
    out = ImageTk.PhotoImage(out)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out

Histogram Equalization

In [67]:
def hist_equalization(channel):
    hist, _ = np.histogram(channel.flatten(), 256, [0, 255])
    cdf = hist.cumsum()
    cdf_norm = ((cdf - cdf.min()) * 255) / (cdf.max() - cdf.min())
    channel_new = cdf_norm[channel.flatten()]
    channel_new = np.reshape(channel_new, channel.shape)
    return channel_new

def enhance():
    im = np.array(image)
    out_r = hist_equalization(im[:,:,0])
    out_g = hist_equalization(im[:,:,1])
    out_b = hist_equalization(im[:,:,2])
    output = np.stack([out_r, out_g, out_b], axis=2)
    output = Image.fromarray(np.array(output).astype(np.uint8), 'RGB')
    out = ImageTk.PhotoImage(output)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out

Film grain

In [68]:
def add_noise_channel(channel, noise):
    channel_noised = channel + noise
    channel_noised = np.clip(channel_noised, 0, 255).astype(np.uint8)
    return channel_noised


def film_grain():
    level = int(GRAIN_LEVEL.get())
    im = np.array(image)
    m = im.shape[0]
    n = im.shape[1]
    noise = np.random.normal(0, level, (m, n))

    out_r = add_noise_channel(im[:,:,0], noise)
    out_g = add_noise_channel(im[:,:,1], noise)
    out_b = add_noise_channel(im[:,:,2], noise)
    output = np.stack([out_r, out_g, out_b], axis=2)

    output = Image.fromarray((output).astype(np.uint8), 'RGB')
    out = ImageTk.PhotoImage(output)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out

Summer filter

In [69]:
def summer():
    im = np.array(image)
    red_level = 0.2
    blue_level = -0.2
    im[:,:,0] = change_intensity(im[:,:,0], red_level)
    im[:,:,2] = change_intensity(im[:,:,2], blue_level)
    output = Image.fromarray(im.astype(np.uint8), 'RGB')
    out = ImageTk.PhotoImage(output)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out

Winter filter

In [70]:
def winter():
    im = np.array(image)
    red_level = -0.2
    blue_level = 0.2
    im[:,:,0] = change_intensity(im[:,:,0], red_level)
    im[:,:,2] = change_intensity(im[:,:,2], blue_level)
    output = Image.fromarray(im.astype(np.uint8), 'RGB')
    out = ImageTk.PhotoImage(output)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out

Pixelate

In [71]:
def pixelate():
    window = int(PIXEL_LEVEL.get())
    im = np.array(image)
    m, n, c = im.shape
    m, n = m - m % window, n - n % window
    out = np.zeros((m, n, c))
    for i in range(0, m, window):
        for j in range(0, n, window):
            out[i: i + window, j : j + window] = im[i: i + window, j : j + window].mean(axis=(0,1))
    out = cv2.resize(out, (im.shape[1], im.shape[0]))
    out = Image.fromarray(np.array(out).astype(np.uint8), 'RGB')
    out = ImageTk.PhotoImage(out)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out


Vintage

In [72]:
def vintage():
    level = int(VINTAGE_LEVEL.get())
    im = np.array(image)
    im = im.astype(np.float64) / 255.0
    h, w = im.shape[:2]

    # create gaussian kernel on each axis
    kernelH = cv2.getGaussianKernel(w, level).reshape([1, -1])
    kernelV = cv2.getGaussianKernel(h, level).reshape([-1, 1])


    kernel = kernelH * kernelV # reshape to [h, w]
    kernel = kernel / kernel.max() # normalize

    # reshape to [h, w, 1]
    kernel = np.expand_dims(kernel, axis=2) 
    # Elementwise multiplication of 
    # [h, w, 3] and [h, w, 1]
    result = np.multiply(im, kernel)
    output = Image.fromarray((result*255).astype(np.uint8), 'RGB')
    out = ImageTk.PhotoImage(output)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out

Threshold

In [73]:
def threshold(img, block_size, c):
    # Check that the block size is odd and nonnegative
    assert block_size % 2 == 1 and block_size > 0, "block_size must be an odd positive integer"
    
    # Calculate the local threshold for each pixel using a Gaussian filter
    threshold_matrix = cv2.GaussianBlur(img, (block_size, block_size), 0)
    threshold_matrix = threshold_matrix - c
    
    # Apply the threshold to the input image
    output = np.zeros_like(img, dtype=np.uint8)
    output[img >= threshold_matrix] = 255
    
    return output

Ink effect

In [74]:
def color_quantization(k=20):

    img = cv2.bilateralFilter(np.array(image), d=7, sigmaColor=200,sigmaSpace=200)
    # Transform the image
    data = np.float32(img).reshape((-1, 3))

    # Determine criteria
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.001)

    # Implementing K-Means
    ret, label, center = cv2.kmeans(data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    center = np.uint8(center)
    result = center[label.flatten()]
    img = np.array(img)
    result = result.reshape((img.shape))

    output = Image.fromarray(result.astype(np.uint8), 'RGB')
    out = ImageTk.PhotoImage(output)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out

Crayon effect

In [75]:
colorsFile = open("ColorCrayon.txt","r")
colors = []
for line in colorsFile.readlines():
    colorset = line.split("\t")
    rgbFormat = [int(x) for x in colorset[2].split(",")]
    colors.append(rgbFormat)


def bitwise_and(channel_1, channel_2):
    m = channel_1.shape[0]
    n = channel_2.shape[1]
    out = np.zeros((m, n))
    for i in range(m):
        for j in range(n):
            if channel_2[i, j] == 0:
                out[i, j] = 0
            else:
                out[i, j] = channel_1[i, j]
    return out

def color_swapping(x,y,rgb_array):
    r1,g1,b1 = rgb_array
    shortest_distance = None
    sol = None
    for color in colors:
        r2,g2,b2 = color
        d = pow(((r2-r1)*0.30),2) + pow(((g2-g1)*0.59),2) + pow(((b2-b1)*0.11),2)
        if(shortest_distance == None or d < shortest_distance):
            shortest_distance = d
            sol = color
    return sol


def crayon_color():
    im = np.array(image)
    im = cv2.bilateralFilter(im, d=7, sigmaColor=200,sigmaSpace=200)
    # convert to grayscale
    img = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    img_thresh = threshold(img, 11, 2)
    # convert back to rgb
    img_thresh = cv2.cvtColor(img_thresh, cv2.COLOR_GRAY2RGB)
    
    # combine threshold image and original image
    out_r = bitwise_and(im[:,:,0],img_thresh[:,:,0])
    out_g = bitwise_and(im[:,:,1],img_thresh[:,:,1])
    out_b = bitwise_and(im[:,:,2],img_thresh[:,:,2])
    output = np.stack([out_r, out_g, out_b], axis=2)

    # generate crayon like color
    width = im.shape[0]
    height = im.shape[1]
    for i in range(0,width):
        for j in range(0,height):
            channels_xy = output[i,j]
            output[i, j] = color_swapping(i,j,channels_xy)
            

    output = Image.fromarray((output).astype(np.uint8), 'RGB')
    out = ImageTk.PhotoImage(output)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out

Pencil Sketch

In [76]:
# Generate Stroke Map
def gen_stroke_map(img, kernel_size, stroke_width=0, num_of_directions=8, smooth_kernel="gauss", gradient_method=0):
    height = img.shape[0] # number of rows, height of the image
    width = img.shape[1] # number of columns, width of the image
    # Let's start with smoothing
    if (smooth_kernel == "gauss"):
        smooth_im = filters.gaussian(img, sigma=np.sqrt(2))
    else:
        smooth_im = filters.median(img) # default is 3x3 kernel size
    # Let's calculate the gradients:
    if not gradient_method:
        # forward gradient: (we pad with zeros)
        imX = np.zeros_like(img)
        diffX = img[: , 1:width] - img[: , 0:width - 1]
        imX[:, 0:width - 1] = diffX
        imY = np.zeros_like(img)
        diffY = img[1:height , :] - img[0:height - 1 , :]
        imY[0:height - 1, :] = diffY
        G = np.sqrt(np.square(imX) + np.square(imY))
    else:
        # Sobel
        sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)
        sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5)
        G = np.sqrt(np.square(sobelx) + np.square(sobely))
    # Let's create the basic line segement (horizontal)
    # make sure it is an odd number, so the lines are at the middle
    basic_ker = np.zeros((kernel_size * 2 + 1, kernel_size * 2 + 1))
    basic_ker[kernel_size + 1,:] = 1 # ------- (horizontal line)
    # Let's rotate the lines in the given directions and perform the classification:
    res_map = np.zeros((height, width, num_of_directions))
    for d in range(num_of_directions):
        ker = transform.rotate(basic_ker, (d * 180) / num_of_directions)
        res_map[:,:, d] = signal.convolve2d(G, ker, mode='same')
    max_pixel_indices_map = np.argmax(res_map, axis=2)
    # What does it compute? every direction has a (height X width) matrix. For every pixel in the matrix,
    # np.argmax returns the index of the direction that holds the pixel with the maximum value
    # and thus we get the max_pixel_indices map is a (height X width) matrix with direction numbers.
    # Now we compute the Classification map:
    C = np.zeros_like(res_map)
    for d in range(num_of_directions):
        C[:,:,d] = G * (max_pixel_indices_map == d) # (max_pixel_indices_map == d) is a binary matrix
    # We should now consider the stroke width before we create S'
    if not stroke_width:
        for w in range(1, stroke_width + 1):
            if (kernel_size + 1 - w) >= 0:
                basic_ker[kernel_size + 1 - w, :] = 1
            if (kernel_size + 1 + w) < (kernel_size * 2 + 1):
                basic_ker[kernel_size + 1 + w, :] = 1
    # It's time to compute S':
    S_tag_sep = np.zeros_like(C)
    for d in range(num_of_directions):
        ker = transform.rotate(basic_ker, (d * 180) / num_of_directions)
        S_tag_sep[:,:,d] = signal.convolve2d(C[:,:,d], ker, mode='same')
    S_tag = np.sum(S_tag_sep, axis=2)
    # Remember that S shpuld be an image, thus we need to make sure the values are in [0,1]
    S_tag_normalized = (S_tag - np.min(S_tag.ravel())) / (np.max(S_tag.ravel()) - np.min(S_tag.ravel()))
    # The last step is to invert it (b->w, w->b)
    S = 1 - S_tag_normalized
    output = Image.fromarray((S*255).astype(np.uint8), 'L')
    return S

# Make sure input image is in [0,1]
def gen_tone_map(img, w_group=0):
    # The first thing we need to do is to calculate the parameters and define weight groups
    w_mat = np.array([[11, 37, 52],
                     [29, 29, 42],
                     [2, 22, 76]])
    w = w_mat[w_group,:]
    # We can now define tone levels like:
    # dark: [0-85]
    # mild: [86-170]
    # bright: [171-255]
    # Assign each pixel a tone level, make 3 lists where each list holds the pixels (indices) of every tone.
    # Use these lists to calculate the parameters for each image.
    
    # For simplicity, we will use the parameters from the paper:
    # for the mild layer:
    u_b = 225
    u_a = 105
    # for the bright layer:
    sigma_b = 9
    # for the dark layer:
    mu_d = 90
    sigma_d = 11
    
    # Let's calculate the new histogram (p(v)):
    num_pixel_vals = 256
    p = np.zeros(num_pixel_vals)
    for v in range(num_pixel_vals):
        p1 = (1 / sigma_b) * np.exp(-(255 - v) / sigma_b)
        if (u_a <= v <= u_b):
            p2 = 1 / (u_b - u_a)
        else:
            p2 = 0
        p3 = (1 / np.sqrt(2 * np.pi * sigma_d)) * np.exp( (-np.square(v - mu_d)) / (2 * np.square(sigma_d)) )
        p[v] = w[0] * p1 + w[1] * p2 + w[2] * p3 * 0.01
    # normalize the histogram:
    p_normalized = p / np.sum(p)
    # calculate the CDF of the desired histogram:
    P = np.cumsum(p_normalized)
    # calculate the original histogram:
    h = exposure.histogram(img, nbins=256)
    # CDF of original:
    H = np.cumsum(h / np.sum(h))
    # histogram matching:
    lut = np.zeros_like(p)
    for v in range(num_pixel_vals):
        # find the closest value:
        dist = np.abs(P - H[v])
        argmin_dist = np.argmin(dist)
        lut[v] = argmin_dist
    lut_normalized = lut / num_pixel_vals
    J = lut_normalized[(255 * img).astype(np.int)]
    # smooth:
    J_smoothed = filters.gaussian(J, sigma=np.sqrt(2))
    return J_smoothed

# Generate Pencil Texture:
def gen_pencil_texture(img, H, J):
    # define the regularization parameter:
    lamda = 0.2
    height = img.shape[0]
    width = img.shape[1]
    # Adjust the input to correspond
#     H_res = transform.resize(H,(height, width))
    H_res = cv2.resize(H, (width, height), interpolation=cv2.INTER_CUBIC)
    H_res_reshaped = np.reshape(H_res, (height * width, 1))
    logH = np.log(H_res_reshaped)
    
#     J_res = transform.resize(J,(height, width))
    J_res = cv2.resize(J, (width, height), interpolation=cv2.INTER_CUBIC)
    J_res_reshaped = np.reshape(J_res, (height * width, 1))
    logJ = np.log(J_res_reshaped)
    
    # In order to use Conjugate Gradient method we need to prepare some sparse matrices:
    logH_sparse = sparse.spdiags(logH.ravel(), 0, height*width, height*width) # 0 - from main diagonal
    e = np.ones((height * width, 1))
    ee = np.concatenate((-e,e), axis=1)
    diags_x = [0, height*width]
    diags_y = [0, 1]
    dx = sparse.spdiags(ee.T, diags_x, height*width, height*width)
    dy = sparse.spdiags(ee.T, diags_y, height*width, height*width)
    
    # Compute matrix X and b: (to solve Ax = b)
    A = lamda * ((dx @ dx.T) + (dy @ dy.T)) + logH_sparse.T @ logH_sparse
    b = logH_sparse.T @ logJ
    
    # Conjugate Gradient
    beta = sparse.linalg.cg(A, b, tol=1e-6, maxiter=60)
    
    # Adjust the result
    beta_reshaped = np.reshape(beta[0], (height, width))
    
    # The final pencil texture map T
    T = np.power(H_res, beta_reshaped)
    
    return T

# It's time to pack it all up (WOOHOOO!)
def pencil_sketch(kernel_size=8, stroke_width=1, num_of_directions=8, smooth_kernel="gauss",
                       gradient_method=0, rgb=False, w_group=0, pencil_texture_path="", stroke_darkness=2, tone_darkness=1.5):
    img = np.array(image)
    if not rgb:
        # Grayscale image:
        im = img
    else:
        # RGB image:
        yuv_img = color.rgb2yuv(img)
        im = yuv_img[:,:,0]
    # Generate the Stroke Map:
    S = gen_stroke_map(im, kernel_size, stroke_width=stroke_width, num_of_directions=num_of_directions,
                       smooth_kernel=smooth_kernel, gradient_method=gradient_method)
    S = np.power(S, stroke_darkness)
    # Generate the Tone Map:
    J = gen_tone_map(im, w_group=w_group)
    
    # Read the pencil texture:
    if not pencil_texture_path:
        pencil_texture = io.imread('./pencils/pencil0.jpg', as_gray=True)
    else:
        pencil_texture = io.imread(pencil_texture_path, as_gray=True)
    # Generate the Pencil Texture Map:
    T = gen_pencil_texture(im, pencil_texture, J)
    T = np.power(T, tone_darkness)
    # The final Y channel:
    R = np.multiply(S, T)
    
    if not rgb:
        return R
    else:
        yuv_img[:,:,0] = R
        out = exposure.rescale_intensity(color.yuv2rgb(yuv_img), in_range=(0, 1))
        output = Image.fromarray((out*255).astype(np.uint8), 'RGB')
        out = ImageTk.PhotoImage(output)
        panelA = Label(image=out, borderwidth=5, relief="sunken")
        panelA.image = out
        panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
        return out

Chalf effect

In [77]:
def sobel_edge_x(im):
    kernel_size = 3
    hx_kernel = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]])
    hy_kernel = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

    m = im.shape[0]
    n = im.shape[1]

    G = np.zeros((m,n))
    Gx = np.zeros((m,n))
    Gy = np.zeros((m,n))
    gx = np.zeros((m,n))
    gy = np.zeros((m,n))
    pad_img = np.pad(im, pad_width=1, mode='constant')

    def get_img_path(i, j):
        img_patch = pad_img[i: i + kernel_size, j: j + kernel_size].T
        return img_patch

    for i in range(m):
        for j in range(n):
            img_patch = get_img_path(i, j)
            gx = np.sum(hx_kernel * img_patch)
            gy = np.sum(hy_kernel * img_patch)
            Gx[i,j] = np.abs(gx)

    return Gx

def sobel_edge_y(im):
    kernel_size = 3
    hx_kernel = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]])
    hy_kernel = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

    m = im.shape[0]
    n = im.shape[1]

    G = np.zeros((m,n))
    Gx = np.zeros((m,n))
    Gy = np.zeros((m,n))
    gx = np.zeros((m,n))
    gy = np.zeros((m,n))
    pad_img = np.pad(im, pad_width=1, mode='constant')

    def get_img_path(i, j):
        img_patch = pad_img[i: i + kernel_size, j: j + kernel_size].T
        return img_patch

    for i in range(m):
        for j in range(n):
            img_patch = get_img_path(i, j)
            gx = np.sum(hx_kernel * img_patch)
            gy = np.sum(hy_kernel * img_patch)         
            Gy[i,j] = np.abs(gy)

    return Gy

In [78]:
def chalk_effect():
    im = np.array(image)

    src = cv2.GaussianBlur(im, (3, 3), 0)
    gray = cv2.cvtColor(src, cv2.COLOR_RGB2GRAY)

    # apply sobel filter
    grad_x = sobel_edge_x(gray)

    # Gradient-Y
    # grad_y = cv.Scharr(gray,ddepth,0,1)
    grad_y = sobel_edge_y(gray)
    
    abs_grad_x = cv2.convertScaleAbs(grad_x)
    abs_grad_y = cv2.convertScaleAbs(grad_y)

    grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)


    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    im_1 = cv2.filter2D(grad, -1, kernel)

    # convert single channel to multi channel
    im_1 = cv2.cvtColor(im_1, cv2.COLOR_GRAY2RGB)

    # resize blackboard -  image
    background = cv2.imread("blackboard.jpg")
    background = cv2.resize(background, (im_1.shape[1], im_1.shape[0]))

    # alpha blending
    dst = cv2.addWeighted(im_1, 0.4, background, 0.8, 30)
    output = Image.fromarray((dst).astype(np.uint8), 'RGB')
    out = ImageTk.PhotoImage(output)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out

Canvas effect

In [79]:
def canvas_effect():
    im = np.array(image)
    blur = cv2.GaussianBlur(im, (3, 3), 0)
    gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)

    # apply sobel filter
    grad_x = sobel_edge_x(gray)

    # Gradient-Y
    # grad_y = cv.Scharr(gray,ddepth,0,1)
    grad_y = sobel_edge_y(gray)


    abs_grad_x = cv2.convertScaleAbs(grad_x)
    abs_grad_y = cv2.convertScaleAbs(grad_y)

    grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)


    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    src1 = cv2.filter2D(grad, -1, kernel)


    # convert single channel to multi channel
    src1 = cv2.cvtColor(src1, cv2.COLOR_GRAY2RGB)


    # experimental - cartoon effect
    # srcy = cv2.GaussianBlur(src, (5, 5), 0)
    sharped = cv2.filter2D(blur, -1, kernel)  # sharp
    # color = cv2.bilateralFilter(srcy, 15, 250, 250)

    # alpha blend images
    e2x = cv2.addWeighted(src1, 0.5, sharped, 0.7, 0)
    output = Image.fromarray((e2x).astype(np.uint8), 'RGB')
    out = ImageTk.PhotoImage(output)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out

Retro effect

In [80]:
def retro_effect():
    im = np.array(image)
    im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    height = im.shape[0]
    width = im.shape[1]

    max_dots = 140
    background_colour = [224, 255, 255] #rgb
    dots_colour = (96,96,96) #rgb
    if height == max(height, width):
        downsized_image = cv2.resize(im, (int(height * (max_dots / width)), max_dots))
    else:
        downsized_image = cv2.resize(im, (max_dots, int(height * (max_dots / width))))
    # image size
    downsized_image_width, downsized_image_height = downsized_image.shape[1], downsized_image.shape[0]

    # increase target image size
    multiplier = 50
    # set size for target image
    blank_img_height = downsized_image_height * multiplier
    blank_img_width = downsized_image_width * multiplier
    # set the padding value so the dots start in frame (rather than being off the edge
    padding = int(multiplier / 2)
    # create canvas containing just the background colour
    blank_image = np.full(
        ((blank_img_height), (blank_img_width), 3), background_colour, dtype=np.uint8
    )
    # prepare for drawing dot-circles on our traget image
    pil_image = Image.fromarray(blank_image)
    draw = ImageDraw.Draw(pil_image)
    downsized_image = np.array(downsized_image)
    # run through each pixel and draw the circle on our blank canvas
    for y in range(0, downsized_image_height):
        for x in range(0, downsized_image_width):
            k = (x * multiplier) + padding
            m = (y * multiplier) + padding
            r = int((0.6 * multiplier) * ((255 - int(downsized_image[y, x])) / 255))
            leftUpPoint = (k - r, m - r)
            rightDownPoint = (k + r, m + r)
            twoPointList = [leftUpPoint, rightDownPoint]
            draw.ellipse(twoPointList, fill=dots_colour)

    pil_image = pil_image.resize((im.shape[1], im.shape[0]))
    out = ImageTk.PhotoImage(pil_image)
    panelA = Label(image=out, borderwidth=5, relief="sunken")
    panelA.image = out
    panelA.grid(row= 1, column=2 , rowspan= 13, columnspan= 5, padx=5, pady=5)
    return out

In [81]:
root = Tk()
root.option_add("*Label.Font", "roboto 12")
style = ttk.Style(root)
style.configure("TButton", font=('roboto', 12))

# root.tk.call('tk', 'scaling', 2.0)
root['bg']='black'
root.title("IMAGE PROCESSING")

l1=Button(master=root, text="UPLOAD",
           fg="white", bg="#292828", width= 80, relief="groove",font =('Verdana', 12), command=upload)
l1.grid(row= 0, column= 2, columnspan= 6, padx=20, pady=20, sticky='nesw')

notebook = ttk.Notebook(root)

notebook.grid(row= 0, column= 1,rowspan=80, columnspan= 1, padx=20, pady=20, sticky='nesw')

basic_edit = Frame(notebook, width=100, height=1000, bg='#292828')
filters = Frame(notebook, width=100, height=1000, bg='#292828')

notebook.add(basic_edit, text="Basic")
notebook.add(filters, text="Filters")


#---------------------------------------------------
L_BRIGTHNESS = Label(basic_edit, text="Brightness",fg="white", bg='#292828')
L_BRIGTHNESS.grid(row=0, column=0)

BRIGTHNESS_LEVEL = Scale(basic_edit, from_=-100, to=100, orient=HORIZONTAL, fg='#292828')
BRIGTHNESS_LEVEL.set(0)
BRIGTHNESS_LEVEL.grid(row=0, column=1)

btn1= Button(basic_edit, text="CHANGE BRIGHTNESS", fg="white", bg="snow4", command=change_brightness)
btn1.grid(row= 2, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')

#---------------------------------------------------
L_CONTRAST = Label(basic_edit, text="Contrast",fg="white", bg='#292828')
L_CONTRAST.grid(row=3, column=0)

CONTRAST_LEVEL = Scale(basic_edit, from_=-100, to=100, orient=HORIZONTAL)
CONTRAST_LEVEL.set(0)
CONTRAST_LEVEL.grid(row=3, column=1)

btn2= Button(basic_edit, text="CHANGE CONTRAST", fg="white", bg="snow4", command=change_contrast)
btn2.grid(row= 4, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')

#---------------------------------------------------
L_SATURATION = Label(basic_edit, text="Saturation",fg="white", bg='#292828')
L_SATURATION.grid(row=5, column=0)

SATURATION_LEVEL = Scale(basic_edit, from_=-100, to=100, orient=HORIZONTAL)
SATURATION_LEVEL.grid(row=5, column=1)

btn3= Button(basic_edit, text="CHANGE SATURATION", fg="white", bg="snow4", command=change_saturation)
btn3.grid(row= 6, column= 0, columnspan=2,padx=10, pady=10, sticky='nesw')

#---------------------------------------------------
L_HUE = Label(basic_edit, text="Hue",fg="white", bg='#292828')
L_HUE.grid(row=7, column=0)

HUE_LEVEL = Scale(basic_edit, from_=-100, to=100, orient=HORIZONTAL)
HUE_LEVEL.set(0)
HUE_LEVEL.grid(row=7, column=1)

btn4= Button(basic_edit, text="CHANGE HUE", fg="white", bg="snow4", command=change_hue)
btn4.grid(row= 8, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')

#---------------------------------------------------
L_BLUR = Label(basic_edit, text="Blur",fg="white", bg='#292828')
L_BLUR.grid(row=9, column=0)

BLUR_LEVEL = Scale(basic_edit, from_=0, to=10, orient=HORIZONTAL)
BLUR_LEVEL.set(0)
BLUR_LEVEL.grid(row=9, column=1)

btn6= Button(basic_edit, text="BLUR", fg="white", bg="snow4", command=filter_blur)
btn6.grid(row= 10, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')

#---------------------------------------------------
btn5= Button(filters, text="SHARPENING", fg="white", bg="snow4", command=sharpen)
btn5.grid(row= 11, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')

#---------------------------------------------------
btn7= Button(filters, text="INVERT", fg="white", bg="snow4", command=inverse_image)
btn7.grid(row= 12, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')

#---------------------------------------------------
L_GRAIN = Label(filters, text="Grain",fg="white", bg='#292828')
L_GRAIN.grid(row=13, column=0)

GRAIN_LEVEL = Scale(filters, from_=0, to=50, orient=HORIZONTAL)
GRAIN_LEVEL.set(0)
GRAIN_LEVEL.grid(row=13, column=1)

btn8= Button(filters, text="FILM GRAIN", fg="white", bg="snow4", command=film_grain)
btn8.grid(row= 14, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')

#---------------------------------------------------
L_PIXEL = Label(filters, text="Pixelate",fg="white", bg='#292828')
L_PIXEL.grid(row=15, column=0)

PIXEL_LEVEL = Scale(filters, from_=0, to=20, orient=HORIZONTAL)
PIXEL_LEVEL.set(0)
PIXEL_LEVEL.grid(row=15, column=1)

btn9= Button(filters, text="PIXELATE", fg="white", bg="snow4", command=pixelate)
btn9.grid(row= 16, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')

#---------------------------------------------------
btn10= Button(filters, text="SUMMER", fg="white", bg="snow4", command=summer)
btn10.grid(row= 17, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')
#---------------------------------------------------
btn11= Button(filters, text="WINTER", fg="white", bg="snow4", command=winter)
btn11.grid(row= 18, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')
#---------------------------------------------------
btn12= Button(filters, text="GRAYSCALE", fg="white", bg="snow4", command=grayscale)
btn12.grid(row= 19, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')
#---------------------------------------------------
btn13= Button(filters, text="SEPIA", fg="white", bg="snow4", command=sepia)
btn13.grid(row= 20, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')
#---------------------------------------------------
btn14= Button(filters, text="INK", fg="white", bg="snow4", command=color_quantization)
btn14.grid(row= 21, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')
#---------------------------------------------------
btn15= Button(filters, text="CRAYOLA", fg="white", bg="snow4", command=crayon_color)
btn15.grid(row= 22, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')
#---------------------------------------------------
btn16= Button(filters, text="PENCIL", fg="white", bg="snow4", command=pencil_sketch)
btn16.grid(row= 23, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')
#---------------------------------------------------
btn18= Button(filters, text="ENHANCE", fg="white", bg="snow4", command=enhance)
btn18.grid(row= 24, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')
#---------------------------------------------------
btn19= Button(filters, text="CHALK", fg="white", bg="snow4", command=chalk_effect)
btn19.grid(row= 25, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')
#---------------------------------------------------
btn20= Button(filters, text="CANVAS", fg="white", bg="snow4", command=canvas_effect)
btn20.grid(row= 26, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')
#---------------------------------------------------
btn20= Button(filters, text="RETRO", fg="white", bg="snow4", command=retro_effect)
btn20.grid(row= 28, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')
#---------------------------------------------------
L_VINTAGE = Label(filters, text="Vintage",fg="white", bg='#292828')
L_VINTAGE.grid(row=29, column=0)

VINTAGE_LEVEL = Scale(filters, from_=0, to=300, orient=HORIZONTAL)
VINTAGE_LEVEL.set(0)
VINTAGE_LEVEL.grid(row=29, column=1)
btn21= Button(filters, text="VINTAGE", fg="white", bg="snow4", command=vintage)
btn21.grid(row= 30, column= 0, columnspan=2, padx=10, pady=10, sticky='nesw')

root.mainloop()